# Test of the clasification pipeline

In [1]:
import os

# Function to change to the parent directory
def change_to_parent_directory():
    # Check if the directory has already been changed
    if not os.environ.get('DIR_CHANGED'):
        try:
            current_dir = os.path.dirname(os.path.abspath(__file__))
        except NameError:
            current_dir = os.getcwd()
        parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
        os.chdir(parent_dir)
        os.environ['DIR_CHANGED'] = '1'
        print(f"Current working directory changed to: {os.getcwd()}")
    else:
        print("Directory has already been changed.")

# Call the function to change the working directory
change_to_parent_directory()

Current working directory changed to: /home/ihranicky/git/domainradar-clf


## Optional: Create testing dataset
Note: If you want do to this, set create_test_parquet to **True**

In [2]:
create_test_parquet = False

if create_test_parquet:
    import pandas as pd
    import numpy as np

    # List of input Parquet files along with their maximum rows and desired labels
    input_files = [
        {'file': 'testdata/misp_2402.parquet', 'max_rows': 500, 'label': 'phishing'},
        {'file': 'testdata/benign_2312.parquet', 'max_rows': 2000, 'label': 'benign'},
        {'file': 'testdata/malware_bp.parquet', 'max_rows': 500, 'label': 'malware'},
        {'file': 'testdata/dga_2310.parquet', 'max_rows': 500, 'label': 'dga'},
    ]

    # Number of rows to select in total
    n_rows = 1000
    
    # Read the first file to get the initial columns and create the first dataframe
    first_file_info = input_files[0]
    combined_df = pd.read_parquet(first_file_info['file'])
    
    # Limit the number of rows if necessary for the first file
    if len(combined_df) > first_file_info['max_rows']:
        combined_df = combined_df.sample(n=first_file_info['max_rows'], random_state=1)
    
    # Overwrite the "label" column with the specified label for the first file
    combined_df['label'] = first_file_info['label']
    
    # Get the columns from the first dataframe
    all_columns = combined_df.columns.tolist()

    # Process the remaining files
    for file_info in input_files[1:]:
        df = pd.read_parquet(file_info['file'])
        
        # Limit the number of rows if necessary
        if len(df) > file_info['max_rows']:
            df = df.sample(n=file_info['max_rows'], random_state=1)
        
        # Overwrite the "label" column with the specified label
        df['label'] = file_info['label']
        
        # Ensure all columns from the first dataframe are present
        for col in all_columns:
            if col not in df.columns:
                df[col] = None
        
        # Align the dataframe to the columns of the first dataframe
        df = df[all_columns]
        
        # Append the dataframe to the combined dataframe
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    # Randomly select n_rows rows from the combined DataFrame
    selected_rows = combined_df.sample(n=n_rows, random_state=1)  # random_state for reproducibility
    
    # Save the selected rows to a new Parquet file
    selected_rows.to_parquet('testdata/sample.parquet')

## Run classification

In [3]:
# Specify the parquet file with the dataset for classification
test_dataset = 'testdata/sample.parquet'

# Number of domain names to classify with each run of the pipeline (0 = classify all)
CHUNK_SIZE = 30

In [4]:
import pandas as pd
import numpy as np
from classifiers.pipeline import Pipeline

# Initialize the classification pipeline
clf = Pipeline()

# Read the input parquet file
input_df = pd.read_parquet(test_dataset)

# Determine the number of chunks
num_chunks = (len(input_df) + CHUNK_SIZE - 1) // CHUNK_SIZE if CHUNK_SIZE > 0 else 1

# Process the dataframe in chunks
for i in range(num_chunks):
    if CHUNK_SIZE > 0:
        start_idx = i * CHUNK_SIZE
        end_idx = start_idx + CHUNK_SIZE
        chunk_df = input_df[start_idx:end_idx]
    else:
        chunk_df = input_df

    # Perform your classification or processing on the working_df here
    print(f"===== Processing chunk {i+1}/{num_chunks} =====")

    chunk_without_label = chunk_df.drop(columns=['label']) # Label should not be known to classifiers
    chunk_results = clf.classify_domains(chunk_without_label)

    for result in chunk_results:
        print(result)
     
    print(f"===== Chunk {i+1}/{num_chunks} completed. =====")


2024-06-04 15:54:30.885834: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-04 15:54:30.885856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-04 15:54:30.886556: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-04 15:54:30.890932: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 15:54:31.463984: W tensorflow/compiler/tf2

===== Processing chunk 1/34 =====
{'domain': 'cocinortemtb.com.do', 'aggregate_probability': 0.0015286133857443929, 'aggregate_description': '...', 'classification_results': [{'classifier': 'Phishing', 'probability': 0.011665243761273184, 'description': 'No phishing detected.', 'details': {'CNN phishing classifier': '0.0%', 'LightGBM phishing classifier': '2.33%'}}, {'classifier': 'Malware', 'probability': 1.1824990906461608e-06, 'description': 'No malware detected.', 'details:': {'CNN malware classifier': '0.0%', 'XGBoost malware classifier': '0.0%'}}, {'classifier': 'DGA', 'probability': 2.1868197563890135e-06, 'description': 'No DGA detected.', 'details:': {}}]}
{'domain': 'beautysalon.melbourne', 'aggregate_probability': 0.000448230974143371, 'aggregate_description': '...', 'classification_results': [{'classifier': 'Phishing', 'probability': 0.5006298073131508, 'description': 'The domain has high level of phishing indicators.', 'details': {'CNN phishing classifier': '100.0%', 'Ligh

## Optional: Generate preliminary results for training the final aggregation classifier

In [5]:
import pandas as pd
import numpy as np
from classifiers.pipeline import Pipeline

# Initialize the classification pipeline
clf = Pipeline()

# Read the input parquet file
input_df = pd.read_parquet(test_dataset)

input_df = input_df.sample(frac=1).reset_index(drop=True)

#preliminary_results_df = clf.generate_preliminary_results(input_df, output_file="test_preliminary_results.parquet")
preliminary_results_df = clf.generate_preliminary_results(input_df, add_final=True)

In [6]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
pd.options.display.float_format = '{:.6f}'.format
preliminary_results_df

,domain_name,dns_available,dns_nonzero,tls_available,tls_nonzero,ip_available,ip_nonzero,rdap_available,rdap_nonzero,geo_available,geo_nonzero,label,phishing_cnn_result,phishing_lgbm_result,malware_cnn_result,malware_xgboost_result,dga_binary_nn_result,phishing_sum,phishing_avg,phishing_prod,malware_sum,malware_avg,malware_prod,total_sum,total_avg,total_prod,badness_probability
0,89875875mnk--inter387ddbnk.repl.co,0.600000,0.200000,1.000000,0.458333,1.000000,0.625000,1.000000,0.708333,1.000000,0.722222,phishing,1.000000,0.999841,1.000000,0.068415,0.000000,1.999841,0.999920,0.999841,1.068415,0.534208,0.068415,3.068256,0.613651,0.000000,0.999980
1,365.rtvslo.si,0.650000,0.325000,1.000000,0.791667,1.000000,0.500000,0.958333,0.791667,1.000000,0.944444,benign,0.000000,0.000219,0.000000,0.001079,0.000005,0.000219,0.000109,0.000000,0.001079,0.000539,0.000000,0.001302,0.000260,0.000000,0.000338
2,chateau-urspelt.lu,1.000000,0.725000,1.000000,0.458333,1.000000,1.000000,0.833333,0.541667,1.000000,1.000000,benign,1.000000,0.010268,1.000000,0.000004,0.035099,1.010268,0.505134,0.010268,1.000004,0.500002,0.000004,2.045371,0.409074,0.000000,0.000095
3,heatpracticallyease.com,0.900000,0.525000,1.000000,0.416667,1.000000,1.000000,1.000000,0.583333,1.000000,1.000000,malware,1.000000,0.965758,0.000000,0.999999,0.464837,1.965758,0.982879,0.965758,0.999999,0.499999,0.000000,3.430593,0.686119,0.000000,0.999996
4,town.nagaizumi.lg.jp,1.000000,0.700000,0.041667,0.000000,1.000000,0.750000,0.958333,0.541667,1.000000,0.666667,benign,1.000000,0.000867,1.000000,0.000002,0.000000,1.000867,0.500433,0.000867,1.000002,0.500001,0.000002,2.000869,0.400174,0.000000,0.000018
5,verify.etrustmark.rs,0.600000,0.200000,1.000000,0.458333,1.000000,0.625000,1.000000,0.583333,1.000000,0.666667,benign,0.000000,0.016481,0.000000,0.003114,0.000000,0.016481,0.008241,0.000000,0.003114,0.001557,0.000000,0.019596,0.003919,0.000000,0.001161
6,eylted.com,0.475000,0.000000,0.041667,0.000000,0.625000,0.000000,0.833333,0.000000,1.000000,0.000000,dga,0.000000,0.004820,0.000000,0.988016,0.937372,0.004820,0.002410,0.000000,0.988016,0.494008,0.000000,1.930207,0.386041,0.000000,1.000000
7,pbndswdswyhdjfn.com,0.900000,0.550000,0.041667,0.000000,1.000000,0.625000,1.000000,0.583333,1.000000,0.666667,malware,1.000000,0.999910,1.000000,0.999998,0.999984,1.999910,0.999955,0.999910,1.999998,0.999999,0.999998,4.999892,0.999978,0.999892,1.000000
8,ujpisogtostnimz.com,0.475000,0.000000,0.041667,0.000000,0.625000,0.000000,0.833333,0.000000,1.000000,0.000000,dga,1.000000,0.003695,1.000000,0.996933,0.986596,1.003695,0.501848,0.003695,1.996933,0.998467,0.996933,3.987225,0.797445,0.003635,1.000000
9,xnx.rest,0.950000,0.625000,1.000000,0.458333,1.000000,0.625000,1.000000,0.708333,1.000000,0.611111,benign,1.000000,0.000146,1.000000,0.000210,0.039843,1.000146,0.500073,0.000146,1.000210,0.500105,0.000210,2.040198,0.408040,0.000000,0.000054


In [9]:
preliminary_results_df[["domain_name", "label", "badness_probability", "phishing_cnn_result", "phishing_lgbm_result", "malware_cnn_result", "malware_xgboost_result", "dga_binary_nn_result"]]

,domain_name,label,badness_probability,phishing_cnn_result,phishing_lgbm_result,malware_cnn_result,malware_xgboost_result,dga_binary_nn_result
0,89875875mnk--inter387ddbnk.repl.co,phishing,0.999980,1.000000,0.999841,1.000000,0.068415,0.000000
1,365.rtvslo.si,benign,0.000338,0.000000,0.000219,0.000000,0.001079,0.000005
2,chateau-urspelt.lu,benign,0.000095,1.000000,0.010268,1.000000,0.000004,0.035099
3,heatpracticallyease.com,malware,0.999996,1.000000,0.965758,0.000000,0.999999,0.464837
4,town.nagaizumi.lg.jp,benign,0.000018,1.000000,0.000867,1.000000,0.000002,0.000000
5,verify.etrustmark.rs,benign,0.001161,0.000000,0.016481,0.000000,0.003114,0.000000
6,eylted.com,dga,1.000000,0.000000,0.004820,0.000000,0.988016,0.937372
7,pbndswdswyhdjfn.com,malware,1.000000,1.000000,0.999910,1.000000,0.999998,0.999984
8,ujpisogtostnimz.com,dga,1.000000,1.000000,0.003695,1.000000,0.996933,0.986596
9,xnx.rest,benign,0.000054,1.000000,0.000146,1.000000,0.000210,0.039843


In [8]:
preliminary_results_df[["domain_name", "label", "badness_probability"]]

,domain_name,label,badness_probability
0,89875875mnk--inter387ddbnk.repl.co,phishing,0.999980
1,365.rtvslo.si,benign,0.000338
2,chateau-urspelt.lu,benign,0.000095
3,heatpracticallyease.com,malware,0.999996
4,town.nagaizumi.lg.jp,benign,0.000018
5,verify.etrustmark.rs,benign,0.001161
6,eylted.com,dga,1.000000
7,pbndswdswyhdjfn.com,malware,1.000000
8,ujpisogtostnimz.com,dga,1.000000
9,xnx.rest,benign,0.000054
